# 📊 Week 10: Evaluation Metrics for ML and RAG Systems

This notebook covers comprehensive evaluation strategies for ML models and RAG systems.

## Table of Contents
1. [Classification Metrics](#1-classification-metrics)
2. [Regression Metrics](#2-regression-metrics)
3. [Ranking Metrics](#3-ranking-metrics)
4. [RAG Evaluation](#4-rag-evaluation)
5. [A/B Testing](#5-ab-testing)
6. [Building an Evaluation Pipeline](#6-building-an-evaluation-pipeline)

---

In [ ]:
# Setup
import sys
sys.path.insert(0, '../..')

import numpy as np
import matplotlib.pyplot as plt
from typing import List, Dict

from src.evaluation import (
    RAGEvaluator,
    MLEvaluator,
    LLMEvaluator,
    BenchmarkRunner,
    EvaluationReport,
)

from src.evaluation.evaluation import (
    accuracy, precision, recall, f1_score, confusion_matrix,
    mean_squared_error, root_mean_squared_error, mean_absolute_error, r2_score,
    mean_reciprocal_rank, precision_at_k, recall_at_k, ndcg_at_k,
)

print("✅ Setup complete!")

---

## 1. Classification Metrics

### 1.1 The Confusion Matrix

```
                    Predicted
                  Pos     Neg
Actual  Pos       TP      FN
        Neg       FP      TN
```

| Metric | Formula | When to Use |
|--------|---------|-------------|
| **Accuracy** | (TP+TN)/All | Balanced classes |
| **Precision** | TP/(TP+FP) | Cost of false positives high |
| **Recall** | TP/(TP+FN) | Cost of false negatives high |
| **F1** | 2·P·R/(P+R) | Need balance |

In [ ]:
# Example: Spam Detection
np.random.seed(42)

# Simulate predictions
y_true = np.array([1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1])
y_pred = np.array([1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0])

# Calculate metrics
acc = accuracy(y_true, y_pred)
prec = precision(y_true, y_pred)
rec = recall(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred)

print("Spam Detection Evaluation")
print("=" * 40)
print(f"Accuracy:  {acc:.3f}")
print(f"Precision: {prec:.3f}")
print(f"Recall:    {rec:.3f}")
print(f"F1 Score:  {f1:.3f}")
print(f"\nConfusion Matrix:")
print(cm)

# Visualize
fig, ax = plt.subplots(figsize=(6, 5))
im = ax.imshow(cm, cmap='Blues')
ax.set_xticks([0, 1])
ax.set_yticks([0, 1])
ax.set_xticklabels(['Not Spam', 'Spam'])
ax.set_yticklabels(['Not Spam', 'Spam'])
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_title('Confusion Matrix')

for i in range(2):
    for j in range(2):
        ax.text(j, i, cm[i, j], ha='center', va='center', fontsize=20)

plt.colorbar(im)
plt.tight_layout()
plt.show()

### 1.2 Precision-Recall Trade-off

In [ ]:
# Simulate predicted probabilities
y_prob = np.array([0.9, 0.2, 0.7, 0.85, 0.3, 0.95, 0.55, 0.1, 0.8, 0.25, 0.88, 0.92, 0.15, 0.05, 0.65])

# Calculate metrics at different thresholds
thresholds = np.arange(0.1, 1.0, 0.1)
precisions = []
recalls = []

for threshold in thresholds:
    y_pred_at_threshold = (y_prob >= threshold).astype(int)
    precisions.append(precision(y_true, y_pred_at_threshold))
    recalls.append(recall(y_true, y_pred_at_threshold))

# Plot
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(thresholds, precisions, 'b-o', label='Precision')
plt.plot(thresholds, recalls, 'r-s', label='Recall')
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Precision-Recall vs Threshold')
plt.legend()
plt.grid(alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(recalls, precisions, 'g-o')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("💡 Key Insight: Higher threshold → Higher precision, Lower recall")

---

## 2. Regression Metrics

| Metric | Formula | Interpretation |
|--------|---------|----------------|
| **MSE** | mean((y - ŷ)²) | Penalizes large errors |
| **RMSE** | √MSE | Same units as target |
| **MAE** | mean(\|y - ŷ\|) | Robust to outliers |
| **R²** | 1 - SS_res/SS_tot | Variance explained |

In [ ]:
# Simulate regression predictions
np.random.seed(42)
y_true_reg = np.array([100, 150, 200, 180, 220, 175, 190, 210, 140, 160])
y_pred_reg = y_true_reg + np.random.normal(0, 15, len(y_true_reg))  # Add noise

# Calculate metrics
mse = mean_squared_error(y_true_reg, y_pred_reg)
rmse = root_mean_squared_error(y_true_reg, y_pred_reg)
mae = mean_absolute_error(y_true_reg, y_pred_reg)
r2 = r2_score(y_true_reg, y_pred_reg)

print("Regression Evaluation (e.g., House Price Prediction)")
print("=" * 50)
print(f"MSE:  {mse:.2f}")
print(f"RMSE: {rmse:.2f} (Average prediction off by ${rmse:.0f}K)")
print(f"MAE:  {mae:.2f}")
print(f"R²:   {r2:.3f} ({r2*100:.1f}% variance explained)")

# Visualize predictions
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.scatter(y_true_reg, y_pred_reg, alpha=0.7)
plt.plot([100, 250], [100, 250], 'r--', label='Perfect')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs Predicted')
plt.legend()
plt.grid(alpha=0.3)

plt.subplot(1, 2, 2)
residuals = y_true_reg - y_pred_reg
plt.hist(residuals, bins=10, edgecolor='black', alpha=0.7)
plt.axvline(0, color='r', linestyle='--')
plt.xlabel('Residual (Actual - Predicted)')
plt.ylabel('Frequency')
plt.title('Residual Distribution')

plt.tight_layout()
plt.show()

---

## 3. Ranking Metrics

For search and recommendation systems.

In [ ]:
# Example: Search Result Evaluation

# Query 1: Relevant docs at positions 1, 3
# Query 2: Relevant doc at position 2
y_true_rank = [[0, 2], [1]]  # Indices of relevant docs
y_pred_rank = [[0, 1, 2, 3, 4], [0, 1, 2, 3, 4]]  # Ranked results

# Calculate MRR
mrr = mean_reciprocal_rank(y_true_rank, y_pred_rank)

print("Search Ranking Evaluation")
print("=" * 40)
print(f"Mean Reciprocal Rank (MRR): {mrr:.3f}")
print()

# Calculate P@K and R@K for single query
y_true_single = [0, 2, 4]  # Relevant
y_pred_single = [0, 1, 2, 3, 4]  # Ranked

for k in [1, 3, 5]:
    p_k = precision_at_k(y_true_single, y_pred_single, k)
    r_k = recall_at_k(y_true_single, y_pred_single, k)
    print(f"P@{k}: {p_k:.3f}, R@{k}: {r_k:.3f}")

In [ ]:
# nDCG Example
y_true_relevance = [3, 2, 1, 0, 0]  # Relevance scores
y_pred_perfect = [0, 1, 2, 3, 4]     # Perfect ranking
y_pred_imperfect = [2, 0, 1, 3, 4]   # Imperfect ranking

ndcg_perfect = ndcg_at_k(y_true_relevance, y_pred_perfect, k=5)
ndcg_imperfect = ndcg_at_k(y_true_relevance, y_pred_imperfect, k=5)

print(f"nDCG@5 (Perfect ranking):   {ndcg_perfect:.3f}")
print(f"nDCG@5 (Imperfect ranking): {ndcg_imperfect:.3f}")

---

## 4. RAG Evaluation

### 4.1 RAG Quality Dimensions

| Dimension | Question | How to Measure |
|-----------|----------|----------------|
| **Faithfulness** | Is answer grounded in context? | Check claims vs context |
| **Relevance** | Does answer address the question? | Semantic similarity |
| **Context Precision** | Was retrieved context useful? | % context actually used |
| **Context Recall** | Did we find all needed info? | Coverage of ground truth |

In [ ]:
# RAG Evaluation
rag_evaluator = RAGEvaluator()

# Test data
questions = [
    "What is machine learning?",
    "How does gradient descent work?"
]

answers = [
    "Machine learning is a subset of AI that enables computers to learn from data.",
    "Gradient descent optimizes by moving in the direction of steepest descent."
]

contexts = [
    ["Machine learning is a branch of artificial intelligence that uses data to learn.",
     "ML algorithms improve through experience."],
    ["Gradient descent is an optimization algorithm.",
     "It iteratively adjusts parameters to minimize loss."]
]

ground_truths = [
    "Machine learning is AI that learns patterns from data.",
    "Gradient descent minimizes loss by following the negative gradient."
]

# Evaluate
results = rag_evaluator.evaluate(questions, answers, contexts, ground_truths)

print("RAG System Evaluation")
print("=" * 40)
for metric, value in results.items():
    if isinstance(value, float):
        print(f"{metric:20s}: {value:.3f}")
    else:
        print(f"{metric:20s}: {value}")

### 4.2 Individual Metric Deep Dive

In [ ]:
# Examine individual metrics
question = "What is deep learning?"
answer = "Deep learning uses neural networks with many layers to learn patterns."
context = "Deep learning is a subset of ML using multi-layer neural networks."

# Faithfulness: Is answer grounded in context?
faithfulness = rag_evaluator.evaluate_faithfulness(answer, [context])
print(f"Faithfulness: {faithfulness:.3f}")

# Relevance: Does answer address the question?
relevance = rag_evaluator.evaluate_relevance(question, answer)
print(f"Relevance: {relevance:.3f}")

print("\n💡 Interpretation:")
print(f"  - Faithfulness {faithfulness:.0%}: Answer is {'well' if faithfulness > 0.7 else 'partially'} grounded")
print(f"  - Relevance {relevance:.0%}: Answer {'addresses' if relevance > 0.7 else 'partially addresses'} the question")

---

## 5. A/B Testing

### 5.1 Statistical Significance

In [ ]:
from scipy import stats

# Simulate A/B test results
np.random.seed(42)

# Model A: baseline CTR = 2%
# Model B: improved CTR = 2.2%
n_samples = 10000
ctr_a = 0.020
ctr_b = 0.022

clicks_a = np.random.binomial(1, ctr_a, n_samples)
clicks_b = np.random.binomial(1, ctr_b, n_samples)

observed_ctr_a = clicks_a.mean()
observed_ctr_b = clicks_b.mean()

# Two-proportion z-test
pooled_ctr = (clicks_a.sum() + clicks_b.sum()) / (2 * n_samples)
se = np.sqrt(pooled_ctr * (1 - pooled_ctr) * (2 / n_samples))
z_score = (observed_ctr_b - observed_ctr_a) / se
p_value = 1 - stats.norm.cdf(z_score)  # One-sided test

print("A/B Test Analysis")
print("=" * 40)
print(f"Model A CTR: {observed_ctr_a:.4f}")
print(f"Model B CTR: {observed_ctr_b:.4f}")
print(f"Lift: {(observed_ctr_b - observed_ctr_a) / observed_ctr_a * 100:.1f}%")
print(f"\nZ-score: {z_score:.3f}")
print(f"P-value: {p_value:.4f}")
print(f"\nResult: {'Significant' if p_value < 0.05 else 'Not significant'} at α=0.05")

---

## 6. Building an Evaluation Pipeline

### 6.1 Complete Evaluation Workflow

In [ ]:
# Complete evaluation pipeline
ml_evaluator = MLEvaluator()

# Classification evaluation
clf_results = ml_evaluator.evaluate_classification(
    y_true=y_true,
    y_pred=y_pred,
    average="macro"
)

# Regression evaluation
reg_results = ml_evaluator.evaluate_regression(
    y_true=y_true_reg,
    y_pred=y_pred_reg
)

# Create report
report = EvaluationReport(title="Model Evaluation Report")
report.add_metrics("Classification", clf_results)
report.add_metrics("Regression", reg_results)
report.add_metrics("RAG", results)

# Display report
print(report.to_markdown())

---

## 📝 Summary

### Choosing the Right Metrics

| Task | Primary Metrics | Consider |
|------|----------------|----------|
| Binary Classification | F1, AUC | Class imbalance |
| Multi-class | Macro F1 | Per-class performance |
| Regression | RMSE, R² | Outlier sensitivity |
| Ranking | nDCG, MRR | Position importance |
| RAG | Faithfulness, Relevance | Context quality |

### Best Practices

1. **Use multiple metrics** - No single metric tells whole story
2. **Match metrics to business goals** - Choose based on costs of errors
3. **Evaluate on held-out data** - Avoid data leakage
4. **Monitor over time** - Watch for drift
5. **A/B test in production** - Validate real-world impact